## Tesla contracts

#### Load Python tools

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import altair as alt
import altair_stiles as altstiles

In [3]:
alt.themes.register("stiles", altstiles.theme)
alt.themes.enable("grid")

ThemeRegistry.enable('grid')

In [4]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000
pd.set_option("display.float_format", lambda x: "%.2f" % x)

### read in data

In [5]:
tesla_duns = "148284255"

In [6]:
tesla_prime = pd.read_csv(
    "data/raw/Contracts_PrimeAwardSummaries_2022-04-29_H16M33S19_tesla.csv",
    dtype={"recipient_parent_duns": str},
)

In [7]:
tesla_prime.total_obligated_amount.sum()

10373.07

In [8]:
tesla_prime_df = (
    tesla_prime[tesla_prime["recipient_parent_duns"] == tesla_duns][
        [
            "award_id_piid",
            "recipient_parent_name",
            "award_description",
            "awarding_agency_name",
            "awarding_sub_agency_name",
            "type_of_contract_pricing",
            "total_obligated_amount",
            "potential_total_value_of_award",
            "award_base_action_date",
            "award_base_action_date_fiscal_year",
        ]
    ]
    .sort_values("total_obligated_amount", ascending=False)
    .copy()
)

In [9]:
tesla_prime_df.total_obligated_amount.sum().round()

10373.0

In [10]:
tesla_prime_df.to_csv("data/processed/tesla_primary_contract_processed.csv")

### read in subawards

In [11]:
tesla_subs = pd.read_csv(
    "data/raw/Contracts_Subawards_2022-04-29_H16M33S24_tesla.csv",
    dtype={"subawardee_parent_duns": str},
)

In [12]:
tesla_subs_df = tesla_subs[tesla_subs["subawardee_parent_duns"] == tesla_duns][
    [
        "subaward_number",
        "subaward_type",
        "subaward_amount",
        "subawardee_parent_duns",
        "subawardee_parent_name",
        "subawardee_name",
        "subaward_action_date",
        "subaward_action_date_fiscal_year",
        "prime_award_awarding_sub_agency_name",
        "prime_awardee_parent_name",
        "prime_awardee_parent_duns",
    ]
].drop_duplicates(subset="subaward_number")

#### Companies

In [13]:
companies = (
    tesla_subs_df.groupby(["prime_awardee_parent_name"])
    .agg({"subaward_amount": sum})
    .reset_index()
    .sort_values("subaward_amount", ascending=False)
)
companies["share"] = (
    companies["subaward_amount"] / companies["subaward_amount"].sum()
) * 100
companies

,prime_awardee_parent_name,subaward_amount,share
0,SAE INTERNATIONAL,705000.00,100.00


#### Agencies

In [14]:
subs_agencies = (
    tesla_subs_df.groupby(["prime_award_awarding_sub_agency_name"])
    .agg({"subaward_amount": sum})
    .reset_index()
    .sort_values("subaward_amount", ascending=False)
)
subs_agencies["share"] = (
    subs_agencies["subaward_amount"] / companies["subaward_amount"].sum()
) * 100
subs_agencies = subs_agencies[subs_agencies["subaward_amount"] > 0]
subs_agencies

,prime_award_awarding_sub_agency_name,subaward_amount,share
0,NATIONAL HIGHWAY TRAFFIC SAFETY ADMINISTRATION,705000.00,100.00


In [15]:
tesla_subs_total = (
    tesla_subs_df.groupby(["subawardee_name"])
    .agg({"subaward_amount": sum})
    .reset_index()
)

#### Years

In [16]:
years = (
    tesla_subs_df.groupby(["subaward_action_date_fiscal_year"])
    .agg({"subaward_amount": sum})
    .reset_index()
    .sort_values("subaward_action_date_fiscal_year", ascending=False)
)
years["share"] = (
    years["subaward_amount"] / tesla_subs_total["subaward_amount"].sum()
) * 100
years = years[years["subaward_amount"] > 0]
years

,subaward_action_date_fiscal_year,subaward_amount,share
0,2013,705000.00,100.00


---

## Export

In [18]:
subs_agencies.to_csv("data/processed/tesla_subcontract_agencies.csv")

In [19]:
companies.to_csv("data/processed/tesla_subcontract_companies.csv")

In [20]:
tesla_subs_df.to_csv("data/processed/tesla_allsubcontracts.csv")

In [21]:
years.to_csv("data/processed/tesla_subcontract_years.csv")

In [22]:
tesla_subs

,prime_award_unique_key,prime_award_piid,prime_award_parent_piid,prime_award_amount,prime_award_disaster_emergency_fund_codes,prime_award_outlayed_amount_funded_by_COVID-19_supplementals,prime_award_obligated_amount_funded_by_COVID-19_supplementals,prime_award_base_action_date,prime_award_base_action_date_fiscal_year,prime_award_latest_action_date,prime_award_latest_action_date_fiscal_year,prime_award_period_of_performance_start_date,prime_award_period_of_performance_current_end_date,prime_award_period_of_performance_potential_end_date,prime_award_awarding_agency_code,prime_award_awarding_agency_name,prime_award_awarding_sub_agency_code,prime_award_awarding_sub_agency_name,prime_award_awarding_office_code,prime_award_awarding_office_name,prime_award_funding_agency_code,prime_award_funding_agency_name,prime_award_funding_sub_agency_code,prime_award_funding_sub_agency_name,prime_award_funding_office_code,prime_award_funding_office_name,prime_award_treasury_accounts_funding_this_award,prime_award_federal_accounts_funding_this_award,prime_award_object_classes_funding_this_award,prime_award_program_activities_funding_this_award,prime_awardee_uei,prime_awardee_duns,prime_awardee_name,prime_awardee_dba_name,prime_awardee_parent_uei,prime_awardee_parent_duns,prime_awardee_parent_name,prime_awardee_country_code,prime_awardee_country_name,prime_awardee_address_line_1,prime_awardee_city_name,prime_awardee_county_name,prime_awardee_state_code,prime_awardee_state_name,prime_awardee_zip_code,prime_awardee_congressional_district,prime_awardee_foreign_postal_code,prime_awardee_business_types,prime_award_primary_place_of_performance_city_name,prime_award_primary_place_of_performance_state_code,prime_award_primary_place_of_performance_state_name,prime_award_primary_place_of_performance_address_zip_code,prime_award_primary_place_of_performance_congressional_district,prime_award_primary_place_of_performance_country_code,prime_award_primary_place_of_performance_country_name,prime_award_description,prime_award_project_title,prime_award_naics_code,prime_award_naics_description,prime_award_national_interest_action_code,prime_award_national_interest_action,subaward_type,subaward_fsrs_report_id,subaward_fsrs_report_year,subaward_fsrs_report_month,subaward_number,subaward_amount,subaward_action_date,subaward_action_date_fiscal_year,subawardee_uei,subawardee_duns,subawardee_name,subawardee_dba_name,subawardee_parent_uei,subawardee_parent_duns,subawardee_parent_name,subawardee_country_code,subawardee_country_name,subawardee_address_line_1,subawardee_city_name,subawardee_state_code,subawardee_state_name,subawardee_zip_code,subawardee_congressional_district,subawardee_foreign_postal_code,subawardee_business_types,subaward_primary_place_of_performance_address_line_1,subaward_primary_place_of_performance_city_name,subaward_primary_place_of_performance_state_code,subaward_primary_place_of_performance_state_name,subaward_primary_place_of_performance_address_zip_code,subaward_primary_place_of_performance_congressional_district,subaward_primary_place_of_performance_country_code,subaward_primary_place_of_performance_country_name,subaward_description,subawardee_highly_compensated_officer_1_name,subawardee_highly_compensated_officer_1_amount,subawardee_highly_compensated_officer_2_name,subawardee_highly_compensated_officer_2_amount,subawardee_highly_compensated_officer_3_name,subawardee_highly_compensated_officer_3_amount,subawardee_highly_compensated_officer_4_name,subawardee_highly_compensated_officer_4_amount,subawardee_highly_compensated_officer_5_name,subawardee_highly_compensated_officer_5_amount,usaspending_permalink,subaward_fsrs_report_last_modified_date
0,CONT_AWD_DTNH2211C00215_6940_-NONE-_-NONE-,DTNH2211C00215,NaN,3032500.00,NaN,NaN,NaN,2011-09-26,2011,2014-08-31,2014,2011-09-26,2014-11-30,2014-11-30,69,Department of Transportation (DOT),6940,NATIONAL HIGHWAY TRAFFIC SAFETY ADMINISTRATION,22,DEPT OF TRANS/NAT HIGHWAY TRAFFIC SAFETY ADM,69,Department of T